In [1]:
import duckdb
import pandas as pd
import yaml
from mosaic_widget import MosaicWidget

In [2]:
# Optional: enable logging in Mosaic widget
import logging
logging.basicConfig(level=logging.INFO)

In [3]:
weather = pd.read_csv("../../data/seattle-weather.csv", parse_dates=['date'])

with open("../specs/weather.yaml") as f:
    spec = yaml.safe_load(f)

In [4]:
MosaicWidget(spec, data = {"weather": weather})

MosaicWidget(spec={'params': {'click': {'select': 'single'}, 'domain': ['sun', 'fog', 'drizzle', 'rain', 'snow…